## Config the model

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
model=ChatGoogleGenerativeAI(model='gemini-2.5-flash-lite')
output=model.invoke("What is the capital of Pakistan?")
print(output.content)

The capital of Pakistan is **Islamabad**.


## Config the embedding model

In [6]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en")
len(embeddings.embed_query("hi"))

c:\Users\dell\miniconda3\envs\RAG\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\dell\.cache\huggingface\hub\models--BAAI--bge-small-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download

384

In [7]:
!pip install hf_xet

   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   --- ------------------------------------ 0.3/2.9 MB ? eta -:--:--
   --- ------------------------------------ 0.3/2.9 MB ? eta -:--:--
   ------- -------------------------------- 0.5/2.9 MB 932.9 kB/s eta 0:00:03
   ---------- ----------------------------- 0.8/2.9 MB 799.2 kB/s eta 0:00:03
   ---------- ----------------------------- 0.8/2.9 MB 799.2 kB/s eta 0:00:03
   ---------- ----------------------------- 0.8/2.9 MB 799.2 kB/s eta 0:00:03
   ---------- ----------------------------- 0.8/2.9 MB 799.2 kB/s eta 0:00:03
   ---------- ----------------------------- 0.8/2.9 MB 799.2 kB/s eta 0:00:03
   ------------------ --------------------- 1.3/2.9 MB 599.2 kB/s eta 0:00:03
   ------------------ --------------------- 1.3/2.9 MB 599.2 kB/s eta 0:00:03
   ------------------------- -------------- 1.8/2.9 MB 724.2 kB/s eta 0:00:02
   -------------------

## lets take a data embedd it and store in VDB

In [8]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [16]:
import os
import shutil

os.makedirs("./data2", exist_ok=True)

# Copy state_of_the_union.txt to data2 directory
source_file = "../state_of_the_union.txt"
dest_file = "./data2/state_of_the_union.txt"

if os.path.exists(source_file):
    shutil.copy2(source_file, dest_file)
    print(f"Copied {source_file} to {dest_file}")
else:
    print(f"Warning: {source_file} not found")

loader = DirectoryLoader("./data2", glob="*.txt", loader_cls=TextLoader)

Copied ../state_of_the_union.txt to ./data2/state_of_the_union.txt


In [17]:
docs=loader.load()

In [18]:
docs

[Document(metadata={'source': 'data2\\state_of_the_union.txt'}, page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their d

In [19]:
docs[0].page_content

'Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. \n\nGroups of citizens blocking tanks with 

In [20]:
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50
)

In [21]:
new_docs=text_splitter.split_documents(documents=docs)

In [22]:
new_docs[0].page_content

'Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.'

In [23]:
doc_string=[doc.page_content for doc in new_docs]

In [24]:
doc_string[0]

'Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.'

In [25]:
len(doc_string)

274

In [27]:
!pip install chromadb

  Using cached build-1.3.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached posthog-5.4.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached pypika-0.48.9-py2.py3-none-any.whl
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached jsonschema-4.25.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
  Using cached pyproject_hooks-1.2.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached jsonschema_specifications-2025.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached durationpy-0.10-py3-none-any.whl.metadata (340 bytes)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached importlib_metadata-8.7.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached zipp-3.23.0-py3-none-a

In [28]:
db=Chroma.from_documents(new_docs,embeddings)

In [29]:
retriever=db.as_retriever(search_kwargs={"k": 3})

In [31]:
retriever.invoke("Madam Vice President?")

[Document(metadata={'source': 'data2\\state_of_the_union.txt'}, page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.'),
 Document(metadata={'source': 'data2\\state_of_the_union.txt'}, page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our'),
 Document(metadata={'source': 'data2\\state_of_the_union.txt'}, page_content='So tonight I’m offering a Unity Agenda for the Nation. Four big things we can do together.  \n\nFirst, beat the opioid epidemic.')]

## creation of pydantic class

In [32]:
# creation of pydantic class
# creation of pydantic class
import operator
from typing import List
from pydantic import BaseModel , Field
from langchain.prompts import PromptTemplate
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph,END

In [33]:
class TopicSelectionParser(BaseModel):
    Topic:str=Field(description="selected topic")
    Reasoning:str=Field(description='Reasoning behind topic selection')

In [34]:
from langchain.output_parsers import PydanticOutputParser

In [35]:
parser=PydanticOutputParser(pydantic_object=TopicSelectionParser)

In [36]:
parser.get_format_instructions()


'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"Topic": {"description": "selected topic", "title": "Topic", "type": "string"}, "Reasoning": {"description": "Reasoning behind topic selection", "title": "Reasoning", "type": "string"}}, "required": ["Topic", "Reasoning"]}\n```'

## This agentstate class you need to inside the stategraph

In [37]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [38]:
state={"messages":["hi"]}

In [39]:
state="hi"

In [40]:
def function_1(state:AgentState):
    
    question=state["messages"][-1]
    
    print("Question",question)
    
    template="""
    Your task is to classify the given user query into one of the following categories: [USA,Not Related]. 
    Only respond with the category name and nothing else.

    User query: {question}
    {format_instructions}
    """
    
    prompt= PromptTemplate(
        template=template,
        input_variable=["question"],
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )
    
    
    chain= prompt | model | parser
    
    response = chain.invoke({"question":question})
    
    print("Parsed response:", response)
    
    return {"messages": [response.Topic]}

In [41]:

state={"messages":["what is a today weather?"]}

In [42]:
state={"messages":["what is a GDP of usa??"]}

In [43]:
function_1(state)

Question what is a GDP of usa??
Parsed response: Topic='USA' Reasoning='The user is explicitly asking about the GDP of the USA, which directly relates to the USA as a topic.'


{'messages': ['USA']}

In [44]:
class TopicSelectionParser(BaseModel):
    Topic:str=Field(description="selected topic")
    Reasoning:str=Field(description='Reasoning behind topic selection')

In [45]:
def router(state:AgentState):
    print("-> ROUTER ->")
    
    last_message=state["messages"][-1]
    print("last_message:", last_message)
    
    if "usa" in last_message.lower():
        return "RAG Call"
    else:
        return "LLM Call"

In [46]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [47]:
# RAG Function
def function_2(state:AgentState):
    print("-> RAG Call ->")
    
    question = state["messages"][0]
    
    prompt=PromptTemplate(
        template="""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:""",
        
        input_variables=['context', 'question']
    )
    
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
    )
    result = rag_chain.invoke(question)
    return  {"messages": [result]}

In [48]:
# LLM Function
def function_3(state:AgentState):
    print("-> LLM Call ->")
    question = state["messages"][0]
    
    # Normal LLM call
    complete_query = "Anwer the follow question with you knowledge of the real world. Following is the user question: " + question
    response = model.invoke(complete_query)
    return {"messages": [response.content]}

In [49]:
from langgraph.graph import StateGraph,END


In [50]:
workflow=StateGraph(AgentState)


In [51]:
workflow.add_node("Supervisor",function_1)


In [52]:
workflow.add_node("RAG",function_2)



In [53]:
workflow.add_node("LLM",function_3)


In [54]:
workflow.set_entry_point("Supervisor")


In [55]:
workflow.add_conditional_edges(
    "Supervisor",
    router,
    {
        "RAG Call": "RAG",
        "LLM Call": "LLM",
    }
)

In [56]:

workflow.add_edge("RAG",END)
workflow.add_edge("LLM",END)

In [57]:
app=workflow.compile()


In [58]:
state={"messages":["hi"]}


In [59]:
app.invoke(state)


Question hi
Parsed response: Topic='Not Related' Reasoning="The query 'hi' is a general greeting and does not contain any information that can be classified as related to the USA."
-> ROUTER ->
last_message: Not Related
-> LLM Call ->


{'messages': ['hi', 'Not Related', 'Hi there! How can I help you today?']}

In [60]:
state={"messages":["what is a gdp of usa?"]}


In [61]:
app.invoke(state)


Question what is a gdp of usa?
Parsed response: Topic='USA' Reasoning='The user is asking about the GDP of the USA, which is a direct query related to the United States of America.'
-> ROUTER ->
last_message: USA
-> RAG Call ->


{'messages': ['what is a gdp of usa?',
  'USA',
  "I'm sorry, but the provided context does not contain information about the GDP of the USA. The text focuses on job creation and economic growth rates."]}

In [62]:
state={"messages":["can you tell me the industrial growth of world's most powerful economy?"]}

In [63]:
state={"messages":["can you tell me the industrial growth of world's poor economy?"]}


In [64]:
result=app.invoke(state)


Question can you tell me the industrial growth of world's poor economy?
Parsed response: Topic='Not Related' Reasoning="The user is asking about the industrial growth of the world's poor economy, which is a global economic question and does not specifically pertain to the USA."
-> ROUTER ->
last_message: Not Related
-> LLM Call ->


In [65]:
result["messages"][-1]


'The industrial growth of the world\'s poor economies is a complex and dynamic issue, and it\'s crucial to understand that there isn\'t a single, uniform experience. However, I can provide you with a general overview of the trends, challenges, and opportunities based on real-world knowledge:\n\n**Key Characteristics of Industrial Growth in Poor Economies:**\n\n*   **Often Starts with Low Value-Added Sectors:** Industrialization in poorer economies typically begins with sectors that require less capital, technology, and skilled labor. This often includes:\n    *   **Agriculture Processing:** Turning raw agricultural products into more refined goods (e.g., milling grain, processing tea, packaging fruits).\n    *   **Textiles and Apparel:** This has historically been a major entry point for industrialization due to relatively low capital requirements and the availability of labor.\n    *   **Light Manufacturing:** Production of basic consumer goods like footwear, simple furniture, and hou

# 📋 Code Summary

## Overview
This notebook demonstrates building a **LangGraph-based Agentic RAG System** that intelligently routes questions to either a RAG pipeline (for USA-related queries) or a general LLM (for other queries).

---

## 🔧 Components

### 1. **Model Configuration**
- **LLM**: Google Gemini 2.5 Flash Lite (`ChatGoogleGenerativeAI`)
- **Embeddings**: HuggingFace BAAI/bge-small-en (384 dimensions)

### 2. **Document Processing Pipeline**
- **Data Source**: `state_of_the_union.txt` (copied to `./data2/`)
- **Text Splitting**: `RecursiveCharacterTextSplitter`
  - Chunk size: 200 characters
  - Chunk overlap: 50 characters
- **Vector Database**: ChromaDB for semantic search
- **Retriever**: Top-k retrieval (k=3)

### 3. **Pydantic Schema**
```python
class TopicSelectionParser(BaseModel):
    Topic: str  # Classification result (USA/Not Related)
    Reasoning: str  # Explanation for classification
```

### 4. **Agent State**
```python
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
```

---

## 🔀 LangGraph Workflow

### **Nodes:**

#### **Supervisor** (`function_1`)
- **Role**: Query classifier
- **Task**: Categorizes user queries into [USA, Not Related]
- **Output**: Parsed topic classification

#### **RAG** (`function_2`)
- **Role**: Retrieval-Augmented Generation
- **Task**: Answers USA-related questions using document context
- **Process**:
  1. Retrieve relevant documents from ChromaDB
  2. Format context
  3. Generate answer using LLM with context

#### **LLM** (`function_3`)
- **Role**: General knowledge answering
- **Task**: Answers non-USA queries using LLM's world knowledge
- **Process**: Direct LLM invocation without retrieval

### **Router** (`router`)
- **Logic**: Routes to RAG if "usa" in topic, else to LLM
- **Conditional branching** based on classification

---

## 📊 Graph Structure

```
START → Supervisor → Router → [RAG or LLM] → END
```

**Edges:**
- Entry: Supervisor
- Conditional: Supervisor → Router → {RAG, LLM}
- Terminal: RAG → END, LLM → END

---

## 🧪 Test Cases

1. **General greeting**: `"hi"` → LLM Call
2. **USA-specific**: `"what is a gdp of usa?"` → RAG Call
3. **Implicit USA reference**: `"industrial growth of world's most powerful economy"` → RAG Call
4. **Non-USA query**: `"industrial growth of world's poor economy"` → LLM Call

---

## 🎯 Key Features

✅ **Intelligent routing** based on query classification  
✅ **Hybrid approach**: Combines retrieval-based and parametric knowledge  
✅ **Structured output** using Pydantic parsers  
✅ **Stateful execution** with LangGraph  
✅ **Modular design** with separate RAG and LLM paths  

---

## 📦 Dependencies

- `langchain-google-genai`
- `langchain-huggingface`
- `langchain-community`
- `chromadb`
- `langgraph`
- `pydantic`

---

## 🚀 Usage Pattern

```python
state = {"messages": ["your question here"]}
result = app.invoke(state)
answer = result["messages"][-1]
```